In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Diabetes/diabetic_data.csv')

In [3]:
dataset.replace('?', np.nan, inplace=True)

In [4]:
dataset.isna().sum().sort_values(ascending=False)

weight                      98569
medical_specialty           49949
payer_code                  40256
race                         2273
diag_3                       1423
diag_2                        358
diag_1                         21
encounter_id                    0
tolazamide                      0
glyburide                       0
tolbutamide                     0
pioglitazone                    0
rosiglitazone                   0
acarbose                        0
miglitol                        0
troglitazone                    0
citoglipton                     0
examide                         0
acetohexamide                   0
insulin                         0
glyburide-metformin             0
glipizide-metformin             0
glimepiride-pioglitazone        0
metformin-rosiglitazone         0
metformin-pioglitazone          0
change                          0
diabetesMed                     0
glipizide                       0
repaglinide                     0
glimepiride   

In [5]:
dataset.drop(['weight','payer_code','medical_specialty'],axis=1,inplace=True)

In [6]:
dataset.dropna(inplace=True)

In [7]:
dataset.drop(['encounter_id','patient_nbr','admission_type_id','discharge_disposition_id','admission_source_id'],axis=1,inplace=True)

In [8]:
dataset

,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1,Caucasian,Female,[10-20),3,59,0,18,0,0,0,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),2,11,5,13,2,0,1,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),2,44,1,16,0,0,0,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,51,0,8,0,0,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,Caucasian,Male,[50-60),3,31,6,16,0,0,0,...,No,Steady,No,No,No,No,No,No,Yes,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),3,51,0,16,0,0,0,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,AfricanAmerican,Female,[80-90),5,33,3,18,0,0,1,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,Caucasian,Male,[70-80),1,53,0,9,1,0,0,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,Caucasian,Female,[80-90),10,45,2,21,0,0,1,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [9]:
dataset.loc[dataset.age== '[0-10)','age'] = 0;
dataset.loc[dataset.age== '[10-20)','age'] = 10;
dataset.loc[dataset.age== '[20-30)','age'] = 20;
dataset.loc[dataset.age== '[30-40)','age'] = 30;
dataset.loc[dataset.age== '[40-50)','age'] = 40;
dataset.loc[dataset.age== '[50-60)','age'] = 50;
dataset.loc[dataset.age== '[60-70)','age'] = 60;
dataset.loc[dataset.age== '[70-80)','age'] = 70;
dataset.loc[dataset.age== '[80-90)','age'] = 80;
dataset.loc[dataset.age== '[90-100)','age'] = 90;
dataset.age = dataset.age.astype(np.int32)


dataset.loc[dataset.max_glu_serum== 'None','max_glu_serum'] = 0;
dataset.loc[dataset.max_glu_serum== 'Norm','max_glu_serum'] = 100;
dataset.loc[dataset.max_glu_serum== '>200','max_glu_serum'] = 200;
dataset.loc[dataset.max_glu_serum== '>300','max_glu_serum'] = 300;
dataset.max_glu_serum = dataset.max_glu_serum.astype(np.int32)


dataset.loc[dataset.A1Cresult== 'None','A1Cresult'] = 0;
dataset.loc[dataset.A1Cresult== 'Norm','A1Cresult'] = 5;
dataset.loc[dataset.A1Cresult== '>7','A1Cresult'] = 7;
dataset.loc[dataset.A1Cresult== '>8','A1Cresult'] = 8;
dataset.A1Cresult = dataset.A1Cresult.astype(np.int32)


dataset.loc[dataset.change== 'No','change'] = 0;
dataset.loc[dataset.change== 'Ch','change'] = 1;
dataset.change = dataset.change.astype(np.int8)



dataset.loc[dataset.diabetesMed== 'No','diabetesMed'] = 0;
dataset.loc[dataset.diabetesMed== 'Yes','diabetesMed'] = 1;
dataset.diabetesMed = dataset.diabetesMed.astype(np.int8)


medications = ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"]

for med in medications:
    dataset.loc[dataset[med] == 'No', med] = -20;
    dataset.loc[dataset[med] == 'Down', med] = -10;
    dataset.loc[dataset[med] == 'Steady', med] = 0;
    dataset.loc[dataset[med] == 'Up', med] = 10;
    dataset[med] = dataset[med].astype(np.int32)
    

categoricals = ['race', 'gender', 'diag_1', 'diag_2', 'diag_3']



for c in categoricals:
    dataset[c] = pd.Categorical(dataset[c]).codes

In [10]:
dataset.loc[dataset.readmitted != 'NO','readmitted'] = 0
dataset.loc[dataset.readmitted == 'NO','readmitted'] = 1

In [11]:
dataset.readmitted = dataset.readmitted.astype(np.int8)

In [12]:
X, y = dataset.drop('readmitted', axis=1), dataset['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# AUTOML

In [38]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

[WARNING] [2023-06-07 15:57:14,057:Client-EnsembleBuilder] No runs were available to build an ensemble from


In [39]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.6321285795871747

In [40]:
import pickle
with open('./models/diabetes_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

# RANDOM FOREST

In [45]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [46]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.6187076772456556

# SVC

In [13]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42)
params = {'C': [0.1, 1, 10]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=5, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 